In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
dataset = pd.read_csv('/Users/tanmayeegupte/Documents/Bankruptcy_Project/Company-Bankruptcy-Prediction/Data.csv')
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [15]:
print("shape of x: {}\nshape of y: {}".format(X.shape,y.shape))

shape of x: (6819, 95)
shape of y: (6819,)


In [16]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [17]:
#defining dataset class
from torch.utils.data import Dataset, DataLoader

In [18]:
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.length = self.x.shape[0]
    
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
    def __len__(self):
        return self.length 
    

In [19]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [20]:
trainset = dataset(X,y)
X.shape,y.shape, X_test.shape,y_test.shape


((5455, 95), (5455,), (1364, 95), (1364,))

In [21]:
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)

In [22]:
#defining the network
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self,input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape,64)
        self.fc2 = nn.Linear(64,1)
#        self.fc3 = nn.Linear(64,1)
    
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
#        x = torch.sigmoid(self.fc3(x))
        return x

In [23]:
#hyper parameters
learning_rate = 0.01
epochs = 1000
model = Net(input_shape=X.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [24]:
#forward loop
losses = []
accur = []
for i in range(epochs):
    for j,(x_train,y_train) in enumerate(trainloader):
    
        #calculate output
        output = model(x_train)
 
        #calculate loss
        loss = loss_fn(output,y_train.reshape(-1,1))
 
        #accuracy
        predicted = model(torch.tensor(X,dtype=torch.float32))
        acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i%50 == 0:
        losses.append(loss)
        accur.append(acc)
        print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.32065531611442566	 accuracy : 0.968102658111824
epoch 50	loss : 0.009221795946359634	 accuracy : 0.970852428964253
epoch 100	loss : 0.006493484601378441	 accuracy : 0.9730522456461962
epoch 150	loss : 0.00590392854064703	 accuracy : 0.9739688359303391
epoch 200	loss : 0.005344007629901171	 accuracy : 0.9752520623281393
epoch 250	loss : 0.004646751563996077	 accuracy : 0.9761686526122824
epoch 300	loss : 0.004155813250690699	 accuracy : 0.9765352887259395
epoch 350	loss : 0.0037975122686475515	 accuracy : 0.9770852428964253
epoch 400	loss : 0.00345267029479146	 accuracy : 0.9778185151237397
epoch 450	loss : 0.00316617707721889	 accuracy : 0.9787351054078827
epoch 500	loss : 0.0027967102359980345	 accuracy : 0.9798350137488543
epoch 550	loss : 0.0025351110380142927	 accuracy : 0.9802016498625115
epoch 600	loss : 0.002356545301154256	 accuracy : 0.9811182401466545
epoch 650	loss : 0.0021625603549182415	 accuracy : 0.982218148487626
epoch 700	loss : 0.001961239380761981	 a

In [25]:
#Testing the model on test data
from sklearn.metrics import confusion_matrix, accuracy_score
predicted1 = model(torch.tensor(X_test,dtype=torch.float32))
y_pred1 = predicted1.reshape(-1).detach().numpy().round()
cm = confusion_matrix(y_test, y_pred1)
print(cm)
accuracy_score(y_test, y_pred1)

[[1305   13]
 [  35   11]]


0.9648093841642229